<a href="https://colab.research.google.com/github/skywalker0803r/baseball_ProofofConcept/blob/main/%E6%A3%92%E7%90%83%E8%BB%8C%E8%B7%A1%E8%BF%BD%E8%B9%A4%E6%A8%A1%E7%B5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 影片取得棒球資料函數

In [2]:
import requests

def get_baseball_trajectory_from_video(video_path: str) -> dict | None:
    """
    將影片檔案發送到棒球軌跡偵測API，
    回傳API回應的JSON資料（成功時），
    失敗或無法解析時回傳None。

    :param video_path: 影片檔案路徑
    :return: API回應的JSON資料或None
    """
    url = "https://baseball-api-gpu-669140972615.us-central1.run.app/predict-video"

    try:
        with open(video_path, "rb") as f:
            files = {"file": (video_path, f, "video/mp4")}
            response = requests.post(url, files=files)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"API呼叫失敗，狀態碼: {response.status_code}")
            print("回應內容:", response.text)
            return None
    except Exception as e:
        print("發送影片或解析回應時發生錯誤:", e)
        return None
response = get_baseball_trajectory_from_video('/content/drive/MyDrive/Baseball Movies/CH_videos_4s/pitch_0001.mp4')
print(response)

{'results': [[0, [565.8875122070312, 580.1090087890625, 694.6997680664062, 591.2822265625]], [1, [566.6984252929688, 579.8909301757812, 694.6730346679688, 591.2117309570312]], [2, [566.38427734375, 579.8252563476562, 694.705078125, 591.2833862304688]], [3, [566.67138671875, 579.7984619140625, 694.58740234375, 591.3233642578125]], [4, [566.9029541015625, 579.7587280273438, 694.606689453125, 591.3743286132812]], [5, [566.1827392578125, 579.8888549804688, 694.8372802734375, 591.6586303710938]], [6, [566.830322265625, 579.883544921875, 694.643798828125, 591.5511474609375]], [7, [566.2115478515625, 579.8626708984375, 695.1761474609375, 591.4466552734375]], [8, [565.7586669921875, 579.764404296875, 694.3702392578125, 591.288330078125]], [9, [564.2806396484375, 579.6780395507812, 694.3939208984375, 591.2296752929688]], [10, [564.0587158203125, 579.6143798828125, 694.3670654296875, 591.20068359375]], [11, [563.7708740234375, 579.5133056640625, 695.6788330078125, 591.1431884765625]], [12, [564.

In [3]:
import json

def save_to_json(data: dict, save_path: str = "baseball_result.json"):
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ 已保存至 {save_path}")
save_to_json(response, "CH_videos_4s_pitch_0001_baseball_result.json")

✅ 已保存至 CH_videos_4s_pitch_0001_baseball_result.json


# 影片渲染棒球軌跡函數

In [4]:
import cv2
import os

def render_baseball_boxes(video_path: str, api_response: dict, output_path: str) -> str:
    """
    在原始影片的每個偵測幀畫上棒球偵測框，輸出新影片檔案。

    :param video_path: 原始影片路徑
    :param api_response: API 回傳的偵測資料，格式如 {'results': [[frame_idx, [x1,y1,x2,y2]], ...]}
    :param output_path: 輸出影片完整路徑（包含檔名及副檔名）
    :return: 輸出影片路徑
    """
    # 確保輸出資料夾存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # 解析偵測結果，轉成 dict: frame_idx -> [x1,y1,x2,y2]
    detections = {frame_idx: box for frame_idx, box in api_response.get('results', [])}

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"無法開啟影片檔: {video_path}")

    # 影片參數
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 建立影片寫入器 (mp4)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 如果此幀有偵測框，畫上方框
        if frame_idx in detections:
            x1, y1, x2, y2 = map(int, detections[frame_idx])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, "Baseball", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()

    return output_path


In [6]:
video_path = "/content/drive/MyDrive/Baseball Movies/CH_videos_4s/pitch_0001.mp4"
output_path = "./CH_videos_4s_pitch_0001.mp4"

output_file = render_baseball_boxes(video_path, response, output_path)
print(f"輸出影片路徑：{output_file}")

輸出影片路徑：./CH_videos_4s_pitch_0001.mp4
